In [1]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader

from datasets import load_metric,load_dataset,Dataset

import transformers
from transformers import AutoTokenizer, DataCollatorWithPadding,RobertaForSequenceClassification,AdamW,get_scheduler,TrainingArguments,Trainer


import itertools
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split,StratifiedKFold
from tqdm.auto import tqdm, trange

import csv
import gc

model_checkpoint = 'roberta-base'

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
transformers.logging.set_verbosity(transformers.logging.ERROR)

BATCH_SIZE = 32

In [2]:
def clean_memory():
    gc.collect()
    torch.cuda.empty_cache()
    
def compute_metrics(testing_dataloader):
    metric = load_metric("f1")

    model.eval()
    for batch in testing_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        metric.add_batch(predictions=predictions, references=batch["labels"])
        
    return metric.compute(average='micro')

In [3]:
data = pd.read_csv('../data/BABE/final_labels_SG2.csv',sep=';')
data = data[['text','label_bias']]
data = data[data['label_bias']!='No agreement']
mapping = {'Non-biased':0, 'Biased':1}
data.replace({'label_bias':mapping},inplace=True)
data.rename(columns={'text':'sentence','label_bias':'label'},inplace=True)

In [4]:
basil_data = pd.read_csv('../data/basil.csv',sep=',')
basil_data_processed = basil_data[basil_data['bias'] == 1][['sentence','bias']]
bd_labels = basil_data_processed['bias']
bd_sentences = basil_data_processed[['sentence']]

In [22]:
skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint);
model = RobertaForSequenceClassification.from_pretrained(model_checkpoint);
model.to(device);

In [23]:
training_args = TrainingArguments(
    output_dir='../',
    num_train_epochs=3,
    per_device_train_batch_size=BATCH_SIZE,
    warmup_steps=0,  
    logging_steps=50,
    disable_tqdm = False,
    save_total_limit=2,
    weight_decay=5e-5)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [8]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [9]:
tokenize = lambda data : tokenizer(data['sentence'], truncation=True)

In [10]:
tokenized_data = Dataset.from_pandas(data)
tokenized_data = tokenized_data.map(tokenize,batched=True)
tokenized_data = tokenized_data.remove_columns(['sentence','__index_level_0__'])
tokenized_data.set_format("torch")

  0%|          | 0/4 [00:00<?, ?ba/s]

In [11]:
tokenized_bd = Dataset.from_pandas(bd_sentences)
tokenized_bd = tokenized_bd.map(tokenize,batched=True)
tokenized_bd = tokenized_bd.remove_columns(['sentence','__index_level_0__'])
#tokenized_bd.set_format("torch")

  0%|          | 0/2 [00:00<?, ?ba/s]

In [24]:
f1_scores = []
k = 100

In [25]:
for train_index, val_index in skfold.split(data[['sentence']],data[['label']]):

    #split for this whole selftraining iteration
    token_train = Dataset.from_dict(tokenized_data[train_index])
    token_valid = Dataset.from_dict(tokenized_data[val_index])
    token_train.set_format("torch")
    token_valid.set_format("torch")
    
    tokenized_bd = Dataset.from_pandas(bd_sentences)
    tokenized_bd = tokenized_bd.map(tokenize,batched=True)
    tokenized_bd = tokenized_bd.remove_columns(['sentence','__index_level_0__'])
    
    #self training
    while True:
        #print("Iteration :",iterations)
        print("Fitting on ", len(token_train), " data")
        #initial training
        model = RobertaForSequenceClassification.from_pretrained(model_checkpoint);
        trainer = Trainer(model,training_args,train_dataset=token_train,data_collator=data_collator,
                          tokenizer=tokenizer);
        trainer.train()
        
        #making predictions on unlabelled dataset
        unlabelled_dataloader = DataLoader(tokenized_bd, batch_size=BATCH_SIZE, collate_fn=data_collator)

        logits = torch.Tensor().to(device)

        model.eval()
        for batch in unlabelled_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            with torch.no_grad():
                outputs = model(**batch)

            logits = torch.cat((logits,F.softmax(outputs.logits)))

        
        #stop when there is not enough of resources
        if len(logits[:,0]) < k or len(logits[:,1]) < k:
            break
            
        #indices of the highest probability ranked predictions
        unbiased_topk_indices = torch.topk(logits[:,0],k)[1]
        biased_topk_indices = torch.topk(logits[:,1],k)[1]


        #insert them into training set
        for index in tqdm(biased_topk_indices):
            item = tokenized_bd[index.item()]
            item['label'] = 1

            token_train = token_train.add_item(item)

        for index in tqdm(unbiased_topk_indices):
            item = tokenized_bd[index.item()]
            item['label'] = 0

            token_train = token_train.add_item(item)

        #remove them from unlabelled
        all_indices = np.arange(0,len(tokenized_bd))
        all_to_drop = torch.cat((biased_topk_indices,unbiased_topk_indices)).cpu()
        remaining = np.delete(all_indices,all_to_drop)

        tokenized_bd = Dataset.from_dict(tokenized_bd[remaining])
    
    #evaluation
    eval_dataloader = DataLoader(token_valid, batch_size=BATCH_SIZE, collate_fn=data_collator)
    f1_scores.append(compute_metrics(eval_dataloader)['f1'])


/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)


  0%|          | 0/2 [00:00<?, ?ba/s]

Fitting on  2938  data


loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.9.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file https://huggingface.co/roberta-base/resolve/main/pytorch_mo

Step,Training Loss
50,0.544700
100,0.465700
150,0.399900
200,0.286100
250,0.239800


...

  0%|          | 0/100 [00:00<?, ?it/s]

Fitting on  4139  data


loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.9.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file https://huggingface.co/roberta-base/resolve/main/pytorch_mo

Step,Training Loss
50,0.614100
100,0.470100
150,0.368900
200,0.300700
250,0.298700
300,0.186000
350,0.189500




Training completed. Do not forget to share your model on huggingface.co/models =)




In [408]:
unlabelled_dataloader = DataLoader(tokenized_bd, batch_size=BATCH_SIZE, collate_fn=data_collator)

In [29]:
np.mean(f1_scores)

0.8184007117833509

In [448]:
biased_topk_indices

tensor([212,  56, 318, 142, 360,  48,  86, 296,  92, 244, 485, 109, 459,  11,
        101, 316, 476, 113, 455, 312,  90, 191, 458, 133, 417, 150, 391,  29,
        388, 383, 163, 204, 229, 143,   6, 102, 307, 248, 121, 430,  10, 464,
        205,  77, 146, 261, 311, 407, 478, 211], device='cuda:0')

In [409]:
logits = torch.Tensor().to(device)
preds = []

model.eval()
for batch in unlabelled_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = torch.cat((logits,F.softmax(outputs.logits)))


/tmp/ipykernel_20586/3050525655.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  logits = torch.cat((logits,F.softmax(outputs.logits)))


##### logits[:,1] is bias label axis, logits[:,0] unbiased

In [302]:
bd_sentences['sentence'].values[1034]

'Every former secretary of the Department of Homeland Security, including former White House chief of staff John Kelly, sent a letter to the president and Congress on Thursday asking them restore the department’s funding.'

In [410]:
unbiased_topk_indices = torch.topk(logits[:,0],int(np.round(len(logits)/10)))[1]
biased_topk_indices = torch.topk(logits[:,1],int(np.round(len(logits)/10)))[1]

In [411]:
biased_topk_indices

tensor([ 379, 1018,  535,  443,  722, 1182,  924, 1187,  301,  402,  398, 1149,
        1148,  155,  871, 1021,  683, 1146,  510,  748,  277, 1022,  170, 1184,
         107,   53,  988,  971, 1081,  567,  928, 1176,  459,  580,  833,  403,
        1154,  538,  995, 1019, 1126, 1147,  570,  865,  572,  654,  638,  332,
         633,  460,  176,  884,  204,  171,  116,  799,   70,  700,  291,  954,
         987,  195,  753,  625,   41,  329,  693, 1091,  146,  184,  453,  902,
          65,  496,   75,  702,  637,  298, 1186, 1083,  903,  664,  369,  773,
         923,  528,   57,  154,  456,  328,  626,  113,  867,  427,   54,  162,
         319,  790,  338,  457,  945,  257,  111,  153,  689, 1014, 1039, 1066,
         245,  438,  343,  854,  597, 1104,  691,  466, 1098,  458,  290,   58,
         157,  394], device='cuda:0')